#1
We ensure reproducibility by fixing a stable project root (baseDir) and storing cleaned data to a canonical processed/ folder. This guarantees that model training always uses the same inputs regardless of editor working directory (e.g. VS Code / Jupyter).

#2
origUserId: the original MovieLens user ID (like “user #196” in their system)

origItemId: the original MovieLens movie ID (like “movie #242”)

rating: what they thought (1–5 stars)
Ratings 4–5 = “I like this, please recommend similar.”

Ratings 1–2 = “I dislike this, please don’t recommend similar.”

timestamp: when it happened (UNIX seconds)

#3
event = a user did something with an item at a time like User 22 watched Movie 377 and gave rating 1 at timestamp 878887116 evrey row is an event 

keep only users with at least 5 ratings and  items with at least 5 ratings

<5 is too weak to learn preference 



Data cleaning/filtering Beforehand
Thought PRocess:
user who only rated 1 movie ever. You can't learn their taste. There's no pattern.

A movie that only one person ever rated. You can't generalize it to others.

If you include extremely cold users/items, negative feedback looks random noise (“I hated this one obscure movie I watched once”). After filtering, negative signals become meaningful (“I systematically don’t like action movies”).

 filtering = denoising.


Recommender models don’t like scattered IDs like 196, 244, 927.
They want dense row indices: 0,1,2,3,…
Same for items.
This is how we’ll later build a user-item matrix or embeddings.
sort by timestamp per user.
TO “train on past, predict the future” split.
Given what you’ve watched so far, would I have guessed your next choice?

Evaluate recommenders by hiding each user's most recent interaction as test and training on their history, and asking the model to recover that heldout item. This simulates predicting the user's next choice based only on what the system would have known at that time.

parquet faster and cleaner than csv , can reload this and train models without redoing all the preprocessing again

Neu:18.01

In [25]:
from pathlib import Path


project_root = Path.cwd().parents[1]

print("Project root:", project_root)
file_path = project_root / "data" / "raw" / "movielens" / "ml-1m" / "ratings.dat"
out_dir = project_root / "data" / "processed" / "movielens" / "ml-1m"
out_dir.mkdir(parents=True, exist_ok=True)

print("Using file:", file_path)
print("Exists:", file_path.exists())

min_user_events = 5
min_item_events = 5


Project root: /home/helin/projects/BachelorThesis/code/srcCode/recsys-negative-feedback
Using file: /home/helin/projects/BachelorThesis/code/srcCode/recsys-negative-feedback/data/raw/movielens/ml-1m/ratings.dat
Exists: True


load raw data

In [12]:
import pandas as pd

ratings = pd.read_csv(
    file_path,
    sep="::",
    engine="python",
    names=["user", "item", "rating", "timestamp"],
)
print("Total rows:", len(ratings))
ratings.head()


Total rows: 1000209


,user,item,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


filter map id split:

In [18]:
import pandas as pd


#5 trying out 10 later
min_user_events = 5
min_item_events = 5

# 1) Filteringg sparse users  items
user_counts = ratings["user"].value_counts()
item_counts = ratings["item"].value_counts()

ratings_f = ratings[
    ratings["user"].isin(user_counts[user_counts >= min_user_events].index)
]
ratings_f = ratings_f[
    ratings_f["item"].isin(item_counts[item_counts >= min_item_events].index)
].copy()

print("After filtering:", len(ratings_f))

# 2) Map 
user_map = {u: i for i, u in enumerate(ratings_f["user"].unique())}
item_map = {it: i for i, it in enumerate(ratings_f["item"].unique())}

ratings_f["user_idx"] = ratings_f["user"].map(user_map)
ratings_f["item_idx"] = ratings_f["item"].map(item_map)

# 3) Sort by time per user
ratings_f = ratings_f.sort_values(["user_idx", "timestamp"])

# 4) Split: last interaction per user 
last_idx = ratings_f.groupby("user_idx")["timestamp"].idxmax()

test = ratings_f.loc[last_idx].copy()
train = ratings_f.drop(last_idx).copy()

print("Train:", len(train), "Test:", len(test))

# 5) Save
train_path = project_root / "data" / "processed" / "movielens" / "ml-1m" / "train_ml1m.parquet"
test_path  = project_root / "data" / "processed" / "movielens" / "ml-1m" /  "test_ml1m.parquet"

train.to_parquet(train_path, index=False)
test.to_parquet(test_path, index=False)

print("Saved:")
print(train_path)
print(test_path)


After filtering: 999611
Train: 993571 Test: 6040
Saved:
/home/helin/projects/BachelorThesis/code/srcCode/recsys-negative-feedback/data/processed/movielens/ml-1m/train_ml1m.parquet
/home/helin/projects/BachelorThesis/code/srcCode/recsys-negative-feedback/data/processed/movielens/ml-1m/test_ml1m.parquet


After filtering: 999611
Train: 993571 Test: 6040
Saved:
/home/helin/projects/BachelorThesis/code/srcCode/recsys-negative-feedback/data/processed/movielens/ml-1m/train_ml1m.parquet
/home/helin/projects/BachelorThesis/code/srcCode/recsys-negative-feedback/data/processed/movielens/ml-1m/test_ml1m.parquet

In [26]:
assert train_path.exists()
assert test_path.exists()
train[["user_idx","item_idx","rating"]].head()


,user_idx,item_idx,rating
31,0,31,4
22,0,22,5
27,0,27,4
37,0,37,5
24,0,24,3


In [ ]:
import pandas as pd
from surprise import Dataset, Reader

train_path = project_root / "data" / "processed" / "movielens" / "ml-1m" / "train_ml1m.parquet"
train_df = pd.read_parquet(train_path)

# Surprise 
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_df[["user_idx", "item_idx", "rating"]], reader)

print("Train rows:", len(train_df))


Train rows: 993571


Train rows: 993571

In [23]:
from surprise import SVD
from surprise.model_selection import cross_validate

algo = SVD(n_factors=50, n_epochs=10, lr_all=0.005, reg_all=0.02)

cross_validate(algo, data, measures=["RMSE", "MAE"], cv=3, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9053  0.9023  0.9027  0.9035  0.0013  
MAE (testset)     0.7166  0.7140  0.7150  0.7152  0.0011  
Fit time          17.86   14.47   14.40   15.58   1.62    
Test time         5.24    4.64    4.16    4.68    0.44    


{'test_rmse': array([0.90533081, 0.90234974, 0.90271072]),
 'test_mae': array([0.71659473, 0.71396815, 0.71495998]),
 'fit_time': (17.859877586364746, 14.46615982055664, 14.399108648300171),
 'test_time': (5.2399773597717285, 4.643237590789795, 4.163722276687622)}

estimate runtime 
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9053  0.9023  0.9027  0.9035  0.0013  
MAE (testset)     0.7166  0.7140  0.7150  0.7152  0.0011  
Fit time          17.86   14.47   14.40   15.58   1.62    
Test time         5.24    4.64    4.16    4.68    0.44    
{'test_rmse': array([0.90533081, 0.90234974, 0.90271072]),
 'test_mae': array([0.71659473, 0.71396815, 0.71495998]),
 'fit_time': (17.859877586364746, 14.46615982055664, 14.399108648300171),
 'test_time': (5.2399773597717285, 4.643237590789795, 4.163722276687622)}

In [24]:
from surprise import SVD
from surprise.model_selection import GridSearchCV

param_grid = {
    "n_factors": [50, 100],
    "n_epochs": [10, 20],
    "lr_all": [0.005, 0.01],
    "reg_all": [0.02, 0.05],
}

gs = GridSearchCV(
    SVD,
    param_grid,
    measures=["rmse"],
    cv=3,
    n_jobs=-1
)

gs.fit(data)

print(gs.best_params["rmse"])
print(gs.best_score["rmse"])


{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05}
0.864717218259104


{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05}
0.864717218259104